설치할 패키지
1.  Transformers
2. accelerate
3. gradio

``` !pip install -q transformers==4.40.0 accelerate gradio```

In [ ]:
# 필요한 module, tokenizer, model calling
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")
model = AutoModelForCausalLM.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")

In [ ]:
prompt = "당신은 유용한 AI 어시스던트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다."
instruction = "한국의 수도는 어디야?"

messages = [
    {"role": "system", "content": f"{prompt}"},  # 프롬프트 설정
    {"role": "user", "content": f"{instruction}"}  # llm에 하는 질문
]

# 토크나이저에 탬플릿 적용하는 것 
input_ids = tokenizer.apply_chat_template( # 받은 메시지를 인코딩하는 부분
    messages,
    add_generation_prompt=True,
    return_tensors="pt"  # pytorch를 이용할 것이라는 의미
).to(model.device)  # 인코딩된 부분을 모델 디바이스로 보냄

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# 모델의 출력을 받는 부분
outputs = model.generate(
    input_ids, # 위에 정의한 인풋 방식
    max_new_tokens=512,  #생성할 토큰의 갯수 = 문장의 길이
    eos_token_id=terminators, # eos 토큰을 terminators에 적용시킴
    do_sample=True,# 샘플링 할 거야
    temperature=1,  # 1에 가까울수록 창의적인 답변
    top_p=0.9,
)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True)) # 디코딩하는 과정

# 아래처럼 작성해도 괜찮으나 imstart와 같은 토큰이 나오게 됨
# decoded = tokenizer.batch_decode(outputs)
# print(decoded[0])